<a href="https://colab.research.google.com/github/aviseth/machine-learning-work/blob/master/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision as thv
import numpy as np
from random import sample
import matplotlib.pyplot as plt
import math

In [ ]:
train = thv.datasets.MNIST('../data', download = True, train = True)
val = thv.datasets.MNIST('../data', download = True, train = False)
print(train.data.shape, len(train.targets))

ntrain = train.data.numpy()
targets = train.targets.numpy()
nval = val.data.numpy()
vtarg = val.targets.numpy()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

torch.Size([60000, 28, 28]) 60000


In [ ]:
ntrain = ntrain[:30000]
targets = targets[:30000]
nval = nval[:5000]
vtarg = vtarg[:5000]

In [ ]:
#ntrain = ntrain.reshape([ntrain.shape[0], -1])
#nval = nval.reshape([nval.shape[0], -1])
        

In [ ]:
def batchcreate(data, shuffle, batchsize):
  batches = math.floor(len(data)/batchsize)
  batches = batches * batchsize
  idx_batch = np.arange(batches)

  if shuffle == True:
    np.random.shuffle(idx_batch)
  idx_batch = idx_batch.reshape(-1, batchsize)
  return idx_batch

In [ ]:
trainbatch = batchcreate(ntrain, True, 32)
print(trainbatch.shape)
valbatch = batchcreate(nval, True, 32)
print(valbatch.shape)

(937, 32)
(156, 32)


In [ ]:
trainbatchtarg = batchcreate(targets, True, 32)
valbatchtarg = batchcreate(vtarg, True, 32)

Using interval sampling to get 50% of the images

In [ ]:
class LinearLayer:
    def __init__(self, dim_in, dim_out):
        self.w = np.random.rand(dim_in, dim_out) - 0.1
        self.b = np.random.rand(1, dim_out) - 0.1
        self.dw = 0
        self.db = 0
        self.cache = 0
        self.z = 0

    #computing Y for inputs prevA and weights and biases,
    #caching the values for backward
    def forward(self, prevA):
        #print(self.w.shape, self.b.shape, prevA.shape)
        z = np.dot(prevA, self.w) + self.b
        self.z = z
        #print(self.z.shape)
        cache = (prevA, self.w, self.b)
        self.cache = cache
        #print(z.shape)
        return z, cache

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward(self, inputval):
      #prevA = self.cache[0]
      #self.w = self.cache[1]
      #self.b = self.cache[2]

      prevA, self.w, self.b = self.cache

      shape_m = prevA.shape[1]
      #dZ = np.multiply(np.dot(inputval, self.w.T), 1 - np.power(self.z, 2))
      
      self.dw = 1/shape_m*np.dot(prevA.T, inputval)
      self.db = 1/shape_m*np.sum(inputval, axis=1, keepdims=True)
      #print("start")
      #print(self.w.shape, self.b.shape, self.dw.shape, self.db.shape)
      #print("end")
      #self.dw = (1 / shape_m) * np.dot(dZ, prevA.T)
      #self.db = (1 / shape_m) * np.sum(dZ, axis=1, keepdims=True)

      #dW = np.dot(dZ, prevA.T)*1/shape_m
      #db = np.sum(dZ, axis = 1, keepdims = True)*1/shape_m
      #dprevA = np.dot(self.W.T, dZ)
      return self.dw, self.db

    def zero_grad(self):
      self.dw, self.db = 0*self.dw, 0*self.db

In [ ]:
class ReLU:
    def forward(self, inputval):
        #print(inputval.shape)
        return np.maximum(0, inputval)
    
    def backward(self, inputval):
        return 1 * (inputval > 0)

    def zero_grad(self):
      None
        #self.dw, self.db = 0*self.dw, 0*self.db

In [ ]:
class SoftmaxCrossEntropy_t:
  def __init__(self):
    self.softmax = None
  
  def forward_sce(self, inputval, onehot):
    expval = np.exp(inputval - inputval.max(axis=1, keepdims=True))
    softmax_op = expval/np.sum(expval,axis=1,keepdims=True)
    self.softmax = softmax_op
    temp = np.argmax(onehot, axis=1)
    temp_new = softmax_op[temp]
    loss=-np.sum(onehot*np.log(temp_new))
    
    return (loss/float(softmax_op.shape[0]), temp)

#derivative of softmax
  def dsoftmax_sce(self, inputval):
    #dexpval = np.exp(inputval-inputval.max(axis=1, keepdims=True))
    #dsoftmax_op = dexpval/np.sum(dexpval, axis=0)*(1-dexpval/np.sum(dexpval, axis = 0))
    #print(inputval.shape)
    #print(self.softmax.shape)
    final = (self.softmax - inputval)/inputval.shape[0]
    return final

  def zero_grad(self):
      None

In [ ]:
l0 = LinearLayer(784, 10)
j = ntrain[0]
j = j.reshape(784,1)
z, _ = l0.forward(j)
print(j.shape)
print(z)

l0.backward(z)

ValueError: ignored

In [ ]:
#init layer
lr = 0.1
l1, l2, l3 = LinearLayer(784, 10), ReLU(), SoftmaxCrossEntropy_t()
net = [l1, l2, l3]

def one_hot(yy):
    b = np.zeros((yy.shape[0], 10))
    b[np.arange(yy.shape[0]),yy.reshape((1,32))] = 1
      

for t in range(1000):
  
  for j in trainbatch:
    #print(ntrain[j].shape)
    x = ntrain[j]
    #print(len(x))
    x = x.reshape(len(x),-1)/255.0
    #print(len(x))
    #print(targets.shape)
    yy = targets[j].reshape(len(x),1)
    #y = y.reshape(len(x),1)
  
    b = np.zeros((yy.shape[0], 10))
    b[np.arange(yy.shape[0]),yy.reshape((1,32))] = 1

  l1.zero_grad()

  #print(x.shape)
  a1, _ = l1.forward(x)
  #print(a1.shape)
  a2 = l2.forward(a1)
  #print("relu")
  #print(a2.shape)
  #onehot = one_hot(yy)
  loss, ypred = l3.forward_sce(a2, b)
  #print(ypred)
  ypred.reshape((-1,1))
  da2 = l3.dsoftmax_sce(b)
  da1 = l2.backward(da2)
  dx = l1.backward(da1)

  dw, db = l1.dw, l1.db

  print(t, loss)

  l1.w = l1.w - lr*dw
  l1.b = l1.b - lr*db

In [ ]:
def validate(w, b):
  loss = 0
  for i in range(0, 5000, 32):
    x, yhat = batcharr, batchtargets

#init layer
lr = 0.1
l1, l2, l3 = LinearLayer(784, 10), ReLU(), SoftmaxCrossEntropy_t()
net = [l1, l2, l3]

def one_hot(yy):
    b = np.zeros((yy.shape[0], 10))
    b[np.arange(yy.shape[0]),yy.reshape((1,32))] = 1
      

for t in range(1000):
  
  for j in valbatch:
    #print(ntrain[j].shape)
    x = nval[j]
    #print(len(x))
    x = x.reshape(len(x),-1)/255.0
    #print(len(x))
    #print(targets.shape)
    yy = vtarg[j].reshape(len(x),1)
    #y = y.reshape(len(x),1)
  
    b = np.zeros((yy.shape[0], 10))
    b[np.arange(yy.shape[0]),yy.reshape((1,32))] = 1

    #l1.zero_grad()

    #print(x.shape)
  a1, _ = l1.forward(x)
  #print(a1.shape)
  a2 = l2.forward(a1)
  #print("relu")
  #print(a2.shape)
  #onehot = one_hot(yy)
  loss, ypred = l3.forward_sce(a2, b)
  #print(ypred)
  ypred.reshape((-1,1))
  #da2 = l3.dsoftmax_sce(b)
  #da1 = l2.backward(da2)
  #dx = l1.backward(da1)

  #dw, db = l1.dw, l1.db

  print(t, loss)

  

0 4.583482720096833
1 4.583482720096833
2 4.583482720096833
3 4.583482720096833
4 4.583482720096833
5 4.583482720096833
6 4.583482720096833
7 4.583482720096833
8 4.583482720096833
9 4.583482720096833
10 4.583482720096833
11 4.583482720096833
12 4.583482720096833
13 4.583482720096833
14 4.583482720096833
15 4.583482720096833
16 4.583482720096833
17 4.583482720096833
18 4.583482720096833
19 4.583482720096833
20 4.583482720096833
21 4.583482720096833
22 4.583482720096833
23 4.583482720096833
24 4.583482720096833
25 4.583482720096833
26 4.583482720096833
27 4.583482720096833
28 4.583482720096833
29 4.583482720096833
30 4.583482720096833
31 4.583482720096833
32 4.583482720096833
33 4.583482720096833
34 4.583482720096833
35 4.583482720096833
36 4.583482720096833
37 4.583482720096833
38 4.583482720096833
39 4.583482720096833
40 4.583482720096833
41 4.583482720096833
42 4.583482720096833
43 4.583482720096833
44 4.583482720096833
45 4.583482720096833
46 4.583482720096833
47 4.583482720096833
48